In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def listify(array):
    """ Helper function to allow np arrays to be turned into json """

    tmp = list()
    
    for arr in array:
        if type(arr) is np.float64 or type(arr) is int or type(arr) is float:
            tmp.append(arr)
        else:
            tmp.append(listify(arr)) # recursively turns arrays into lists
    
    return tmp

In [ ]:
realdata = pd.read_csv(f'./Raw/Brownlow New/.csv')

In [ ]:
accur = f'score'
name = f''

In [5]:
attributes = list(realdata.columns)[:-10]


real_vals = list()
for attribute in attributes:
    real_vals.append(sorted(list(set(realdata[attribute]))))


num_arg_vals = [len(real_vals[i]) for i in range(len(real_vals))]



coord_builder = dict()
for i in range(len(attributes)):
    attribute = attributes[i]

    tmp_dict = dict()
    for j in range(len(real_vals[i])):
        tmp_dict[real_vals[i][j]] = j

    coord_builder[attribute] = tmp_dict


synth_data = np.zeros(num_arg_vals)


best_score = -np.inf
for index, data in realdata.iterrows():

    coord = list()

    for attribute in attributes:
        coord.append(coord_builder[attribute][data.loc[attribute]])
    
    if data.loc[accur] > best_score:
        best_score = data.loc[accur]
        best_coord = coord

    synth_data[tuple(coord)] = data.loc[accur]


json_output = {
    'num_arg_vals': num_arg_vals,
    'synthetic_data': listify(synth_data),
    'max': {'synth_max': best_score, 'synth_max_coord': best_coord},
}


with open(f'../Batch 4/{name}.json', 'w') as f:
    json.dump(json_output, f, indent=2)